In [35]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [31]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
pinpir       13
boardname    esp8266sens4
wifialt      DoESLiverpool,decafbad00,10.0.100.1

Sent 7 lines (180 bytes) to config.txt.


In [36]:
%sendtofile --quiet --source utils.py
%sendtofile --quiet --source stdmqttas.py

[leftinbuffer] ['ip:10.0.35.209,mask:255.0.0.0,gw:10.0.0.1']
Sent 32 lines (969 bytes) to utils.py.
Sent 33 lines (1130 bytes) to stdmqttas.py.


In [ ]:
#%sendtofile main.py

import time
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask
from mqtt_as import config, MQTTClient

pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
pinpir = Pin(int(fconfig["pinpir"]), Pin.IN)
boardname = fconfig["boardname"]    
topicstatus = boardname+"/ip"

npircycles = const(20)
async def timesincetask(pin, topic):
    t0 = time.ticks_ms()
    while True:
        for i in range(npircycles):
            if i < 2:
                pinled.value(1-pinled.value())
            await asyncio.sleep_ms(100)
            if pin.value():
                t0 = time.ticks_ms()
        v = (time.ticks_ms() - t0)/1000
        print(topic, v)
        pinled.value(v<10)
        await client.publish(topic, "%.3f"%v)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, ipnumber, retain=True)

config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(mqttconnecttask(client))
aloop.create_task(timesincetask(pinpir, boardname+'/pirseconds'))
aloop.run_forever()



sleep disable
connecting to: DoESLiverpool
ipnumber 10.0.35.209
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.001
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0.0
.esp8266sens4/pirseconds 0.0
esp8266sens4/pirseconds 0

In [ ]:
# original code that handled radar and noisemic

In [13]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
pinnoise     12
pinpir       13
pinradar     14
boardname    esp8266sens3

Sent 8 lines (163 bytes) to config.txt.


In [5]:
# SCK   D5  GPIO14
# MISO  D6  GPIO12  
# MOSI  D7  GPIO13
# PIR module from above pins on bottom is [+ output gnd]


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [14]:
import os
os.remove("main.py")

In [11]:
%sendtofile main.py

import time
from machine import Pin, I2C
from mqtt_as import config
from mqtt_as_timeout import MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)

pinnoise = Pin(int(fconfig["pinnoise"]), Pin.IN)
pinpir = Pin(int(fconfig["pinpir"]), Pin.IN)
pinradar = Pin(int(fconfig["pinradar"]), Pin.IN)

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"].encode()    

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 1600)
        pinled.value(1)
        if pinpir.value():
            await asyncio.sleep_ms(100)
            pinled.value(0)
            await asyncio.sleep_ms(20)
            pinled.value(1)
        await asyncio.sleep_ms(2000)

        
nnoisesamples = const(1000)        
nnoisecycles = const(10)
async def noisetask(topic):
    while True:
        noiseseq = [ ]
        for i in range(nnoisecycles):
            await asyncio.sleep_ms(100)
            v = (sum(pinnoise.value() for i in range(nnoisesamples)))
            noiseseq.append(v)
        print(noiseseq)
        v = sum(noiseseq)/(nnoisesamples*nnoisecycles)
        await client.publish(topic, "%.3f"%v)


npircycles = const(20)
async def timesincetask(pin, topic):
    t0 = time.ticks_ms()
    while True:
        for i in range(npircycles):
            await asyncio.sleep_ms(100)
            if pin.value():
                t0 = time.ticks_ms()
        v = (time.ticks_ms() - t0)/1000
        print(topic, v)
        await client.publish(topic, "%.3f"%v)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+b'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(ledtask())
aloop.create_task(timesincetask(pinpir, boardname+b'/pirseconds'))
aloop.create_task(timesincetask(pinradar, boardname+b'/radarseconds'))
#aloop.create_task(noisetask(boardname+b'/noiselevel'))
aloop.create_task(mqtttask())
aloop.run_forever()





Sent 93 lines (2931 bytes) to main.py.
